 ### <center>     <center>Практическое задание 2. Урок 2. Парсинг HTML. BeautifulSoup, MongoDB
    
<font size=3>  <center>
<font size=3>  <center>**Модуль: "Методы сбора и обработки данных из сети Интернет"**
    
     ------------------------------------------------------------------------------------------------------------------

##### <font size=2 color=blue> Вариант 1
<font size=2 color=blue>  Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). <br>
    Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:<br>
Наименование вакансии.<br>
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).<br>
Ссылку на саму вакансию.<br>
Сайт, откуда собрана вакансия.<br>
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). <br>Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.<br> Сохраните в json либо csv.

In [2]:
# импортируем библиотеки
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
import pandas as pd

In [3]:
def parser_hh(vacancy_name):

    url = 'https://hh.ru'
    full_url = url+'/search/vacancy'
    vacancy = vacancy_name
    
#     зададим параметры поиска и заголовки
    params = {'text': vacancy
              }
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 YaBrowser/21.8.3.614 Yowser/2.5 Safari/537.36'}
    
    vacancy_data = []
        
    while True:
        response = requests.get(full_url, headers=headers, params=params)

#       получим ДОМ
        soup = bs(response.text, 'html.parser')

#       найдем блоки, содержащие сведения по вакансии
        vacancy_list = soup.find_all('div', attrs={'class': 'vacancy-serp-item'})

#     из массива блоков вакансий возьмем каждый и выделим интересующие нас поля 
        for vacant in vacancy_list:
            vacant_dict = {}
#           наименование вакансии
            vacant_name = vacant.find('div', attrs={'class': 'vacancy-serp-item__info'}).text
#           ссылка на вакансию
            vacant_link = vacant.find('a', attrs={'class': 'bloko-link'})['href']
#           город вакансии
            vacant_town = vacant.find('span', attrs={'class': 'vacancy-serp-item__meta-info'}).text
#           организация
            vacant_plant = vacant.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText()
            # чистим поле организации от мусора
            if vacant_plant.find('\xa0') != -1:
                vacant_plant = vacant_plant.replace('\xa0',' ')

            vacant_dict['Наименование'] = vacant_name
            vacant_dict['Ссылка'] = vacant_link
            vacant_dict['Расположение'] = vacant_town
            vacant_dict['Организация'] = vacant_plant

#           зарплата по вакансии: почистим и разнесем по полям "мин" "макс" и "валюта"
            try:
                vacant_salary = vacant.find('div', attrs={'class': 'vacancy-serp-item__sidebar'}).text
                if vacant_salary.find('\u202f') != -1:
                    vacant_salary = vacant_salary.replace('\u202f', ' ')
                if vacant_salary == ' ' or vacant_salary == '':
                    vacant_salary = 'Nan'

            except:
                vacant_salary = 'Nan'

            if vacant_salary != 'Nan':
                vacant_salary = re.split(r'\s|-', vacant_salary)
                if vacant_salary[0] == 'до':
                    salary_min = None
                    salary_max = int(vacant_salary[1])*1000
                elif vacant_salary[0] == 'от':
                    salary_min = int(vacant_salary[1])*1000
                    salary_max = None
                else:
                    salary_min = int(vacant_salary[0])*1000
                    salary_max = int(vacant_salary[3])*1000

                salary_currency = vacant_salary[len(vacant_salary)-1]

                vacant_dict['з/п мин.'] = salary_min
                vacant_dict['з/п макс.'] = salary_max
                vacant_dict['Валюта'] = salary_currency

            else:
                
                vacant_dict['з/п мин.'] = 'Nan'
                vacant_dict['з/п макс.'] = 'Nan'
                vacant_dict['Валюта'] = 'Nan'

            vacant_dict['site'] = 'hh.ru'
            vacancy_data.append(vacant_dict)
# организовываем переход на следующую страницу

        next = soup.find('a', attrs={'data-qa': 'pager-next'})
        if next:
            full_url = url + next['href']
        else:
            break

    df = pd.DataFrame(vacancy_data)
    
    return df

In [21]:
def parser_sj(vacancy_name):

    url = 'https://www.superjob.ru'
    full_url = url+'/vacancy/search/'

    vacancy = vacancy_name
    page = 1
    params = {'keywords': vacancy,
              'noGeo': 1,
              'page': page
              }

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 YaBrowser/21.8.3.614 Yowser/2.5 Safari/537.36'}

    vacancy_data = []
   
    while True:

        response = requests.get(full_url, headers=headers, params=params)
        soup = bs(response.text, 'html.parser')
        vacancy_list = soup.find_all('div', attrs={'class': 'f-test-vacancy-item'})

        if not vacancy_list or not response.ok:
            break
                
        for vacant in vacancy_list:
            vacant_dict = {}
            vacant_block = vacant.find('div', attrs={'class': '_1h3Zg _2rfUm _2hCDz _21a7u'})
            vacant_name = vacant_block.text
            vacant_link = url + vacant_block.find('a', attrs={'class': 'icMQ_'})['href']
            
            vacant_dict['Наименование'] = vacant_name
            vacant_dict['Ссылка'] = vacant_link
            
            try:
                vacant_plant = vacant.find('span', attrs={'class': 'f-test-text-vacancy-item-company-name'}).text
            except:
                vacant_plant = 'Nan'

            vacant_town = vacant.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[2].text
            
            vacant_dict['Расположение'] = vacant_town
            vacant_dict['Организация'] = vacant_plant
            
            try:
                vacant_salary = vacant.find('span', attrs={'class': '_1h3Zg'}).text
                if vacant_salary.find('\xa0') != -1:
                    vacant_salary = vacant_salary.replace('\xa0', ' ')
                    vacant_salary = re.split(r'\s|-', vacant_salary)

                    if vacant_salary[0] == 'до':
                        salary_min = 'Nan'
                        salary_max = int(vacant_salary[1])*1000
                    elif vacant_salary[0] == 'от':
                        salary_min = int(vacant_salary[1])*1000
                        salary_max = 'Nan'
                    elif vacant_salary[0] == 'По':
                        salary_min = 'Nan'
                        salary_max = 'По договоренности'
                    else:
                        salary_min = int(vacant_salary[0])*1000
                        salary_max = int(vacant_salary[3])*1000
                salary_currency = vacant_salary[len(vacant_salary)-1]
                
                vacant_dict['з/п мин.'] = salary_min
                vacant_dict['з/п макс.'] = salary_max
                vacant_dict['Валюта'] = salary_currency

            except:
                vacant_salary = 'Nan'

            vacant_dict['site'] = "superjob.ru"

            vacancy_data.append(vacant_dict)
        
        params['page'] += 1


    df = pd.DataFrame(vacancy_data)
    
    return df

In [22]:
# функция объединения двух поисков в один датафрейм

def parser_twin(vacancy):
        
    df_hh = parser_hh(vacancy)
    df_sj = parser_sj(vacancy)
    
    df = pd.concat([df_hh, df_sj], ignore_index=True)

    return df

In [26]:
# vacancy_name = 'Data Scientist' 'Python'

vacancy_name = 'Data Scientist'
df_vacancy = parser_twin(vacancy_name)

In [27]:
df_vacancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Наименование  320 non-null    object
 1   Ссылка        320 non-null    object
 2   Расположение  320 non-null    object
 3   Организация   320 non-null    object
 4   з/п мин.      298 non-null    object
 5   з/п макс.     292 non-null    object
 6   Валюта        305 non-null    object
 7   site          320 non-null    object
dtypes: object(8)
memory usage: 20.1+ KB


In [28]:
df_vacancy.tail(50)

,Наименование,Ссылка,Расположение,Организация,з/п мин.,з/п макс.,Валюта,site
270,UX Designer,https://hh.ru/vacancy/48059626?from=vacancy_se...,Москва,ИБМ Восточная Европа/Азия - IBM EE/A,Nan,Nan,Nan,hh.ru
271,Риск-аналитик,https://hh.ru/vacancy/47219769?from=vacancy_se...,Казань,ANCOR,Nan,Nan,Nan,hh.ru
272,Риск-аналитик,https://hh.ru/vacancy/47219770?from=vacancy_se...,Нижний Новгород,ANCOR,Nan,Nan,Nan,hh.ru
273,Риск-аналитик,https://hh.ru/vacancy/47219771?from=vacancy_se...,Уфа,ANCOR,Nan,Nan,Nan,hh.ru
274,Риск-аналитик,https://hh.ru/vacancy/47219768?from=vacancy_se...,Самара,ANCOR,Nan,Nan,Nan,hh.ru
275,Data Engineer (SQL),https://hh.ru/vacancy/47942914?from=vacancy_se...,Волгоград,Nitka Technologies,Nan,Nan,Nan,hh.ru
276,Риск-аналитик,https://hh.ru/vacancy/47219774?from=vacancy_se...,Воронеж,ANCOR,Nan,Nan,Nan,hh.ru
277,Ведущий инженер больших данных (Lead Data),https://hh.ru/vacancy/47967136?from=vacancy_se...,Санкт-Петербург,АО Северсталь. Управляющая компания,Nan,Nan,Nan,hh.ru
278,Риск-аналитик,https://hh.ru/vacancy/47219773?from=vacancy_se...,Санкт-Петербург,ANCOR,Nan,Nan,Nan,hh.ru
279,Data Engineer (SQL),https://hh.ru/vacancy/47942733?from=vacancy_se...,Санкт-Петербург,Nitka Technologies,Nan,Nan,Nan,hh.ru
